In [12]:
import PyPDF2
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from langchain.schema import Document
import torch
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer, LlamaConfig
from langchain_text_splitters import RecursiveCharacterTextSplitter
import PyPDF2
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [13]:
# --- LLM Model Loading ---
torch_dtype = torch.bfloat16
model_id = "llm4fun/vietrag-7b-v1.0"
device = "cuda"

tokenizer = LlamaTokenizer.from_pretrained(model_id)
model = LlamaForCausalLM.from_pretrained(
    model_id,
    config=LlamaConfig.from_pretrained(model_id),
    torch_dtype=torch_dtype
)
model = model.eval().to(device)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.42it/s]


In [14]:
# Function to extract text from a PDF file using PyPDF2
def extract_text_from_pdf(pdf_path):
    corpus = []
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text = page.extract_text()
            if text:  # Ensure there's text on the page
                corpus.append(text.strip())
    return corpus

# Path to your PDF file
pdf_path = "/home/jupyter-trunglph/Others/RAG/21120157_21120415.pdf"

# Extract the corpus from the PDF
raw_texts = extract_text_from_pdf(pdf_path)

# Convert extracted text to Document objects
docs = [Document(page_content=text) for text in raw_texts]
docs

[Document(metadata={}, page_content='ĐẠI HỌC QUỐC GIA THÀNH PHỐ HỒ CHÍ MINH\nTRƯỜNG ĐẠI HỌC KHOA HỌC TỰ NHIÊN\nKHOA CÔNG NGHỆ THÔNG TIN\nTRÍ TUỆ NHÂN TẠO CHO AN NINH THÔNG TIN\nBÁO CÁO BÀI TẬP THỰC HÀNH\nLab 4 - Bảo mật trong Quá trình Huấn luyện\nMô hình Máy học\nThành viên:\n21120157 - Lê Phạm Hoàng Trung\n21120415 - Trần Ngọc Bảo\nKhoa: Công nghệ Thông tin\nThành phố Hồ Chí Minh - 2024'),
 Document(metadata={}, page_content='Trường Đại học Khoa học Tự Nhiên TP.HCM Khoa: Công nghệ Thông tin\nMục lục\nI Thông tin chung 4\nII Nội dung báo cáo 5\n1 Mô hình gốc ban đầu 5\n1.1 Tổng quan về Federated Learning . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5\n1.2 Cấu trúc mô hình sử dụng ResNet-18 trong FL . . . . . . . . . . . . . . . . . . . . . . 5\n1.3 Quy trình hoạt động của máy chủ (Server) . . . . . . . . . . . . . . . . . . . . . . . . 5\n1.4 Quy trình hoạt động của máy khách (Client) . . . . . . . . . . . . . . . . . . . . . . . 6\n1.5 Tổng hợp mô hình . . . . . . . . . 

In [15]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

In [16]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Wrap the embedding model
def embed_documents(texts):
    return embedding_model.encode(texts, convert_to_tensor=True).cpu().numpy()
  
# Initialize a FAISS index
dimension = embedding_model.get_sentence_embedding_dimension()
index = faiss.IndexFlatL2(dimension)

In [17]:
vector_store = FAISS(
    embedding_function=embed_documents,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

vector_store.add_documents(documents=all_splits)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


['3ed7e692-2756-4fcc-99a4-50d5917e702a',
 '997d6f70-e213-4a10-b482-19b995c00d98',
 '2fa0eef2-930f-4912-88f1-057e9f06e8db',
 '9f1df2a4-33a4-4aa3-b651-3ce71fa2f25a',
 '8777799d-fbcf-4d2b-bb20-b926bcd5a9c9',
 '734b3439-24fc-47bd-8343-0079347d5703',
 '0e6a9067-bb44-473f-afa5-45ddc5889086',
 '458ce342-0610-4b11-8cd5-8e6da7c9a28f',
 '0e501bdb-b5a4-48e2-9c45-11950320fbd0',
 '5f59a1a0-6302-447e-8ceb-293f7c0d79cf',
 '8717cfa7-8753-4a1f-bf1c-16bf24d35b4d',
 '503ec3fd-78a3-46bd-b5c0-232fc779085a',
 '20eb483c-3340-49e9-9d0a-83e20c6b3a72',
 'b143a4f2-ae99-4382-86b7-980c2006b876',
 '81e25ed1-24b8-43d1-b1ae-d9cae74e9f94',
 'e80e8904-3ba4-424e-a832-33dc4a236d83',
 '4f9d2448-29ed-48e1-8ee3-ab536f0043cb',
 'f93c4343-756f-416d-8282-ab9e44e8dc04',
 'b61fd387-0e1c-4be0-ae97-b3c9617a8460',
 '236d367c-0477-4b56-a279-7f4e6004430f',
 '6141209c-295d-4cd2-8814-a2fb476e63b7',
 '5e0bfb63-d083-4edf-ab8d-475a5db1bb14',
 'c2863f64-2d7a-42fb-b5ba-f59435854e74',
 '1f1d8844-cfc5-47c9-8bb9-a5f60465afb2',
 '2a3e2ef5-13a1-

In [18]:
# --- Prompt Construction ---
prompt_template = (
  "### System:\n"
  "Below is an instruction that describes a task, paired with an input that provides further context. "
  "Write a response that appropriately completes the request.\n\n\n\n"
  "### Instruction:\n{instruction}\n\n"
  "### Input:\n{input}\n\n"
  "### Response:\n{output}"
)

def get_prompt(question, contexts):
  """Generate a prompt for the language model."""
  context = "\n\n".join([f"Context [{i+1}]: {x.page_content}" for i, x in enumerate(contexts)])
  instruction = 'You are an AI assistant. Provide a detailed answer so user don’t need to search outside to understand the answer.'
  input = f"Dựa vào một số ngữ cảnh được cho dưới đây, trả lời câu hỏi ở cuối.\n\n{context}\n\nQuestion: {question}\nHãy trả lời chi tiết và đầy đủ."
  prompt = prompt_template.format(
      instruction=instruction,
      input=input,
      output=''
  )
  return prompt

In [19]:
# --- Vector Search with FAISS ---
def retrieve(question, topk=3):
  """
  Retrieve the most relevant passages using FAISS and vector similarity.
  """

  results = vector_store.similarity_search(question, k=topk)
  return results

In [20]:
def generate(prompt, max_new_tokens=1024):
  """
  Generate a text response from the language model using the provided prompt.
  """
  input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(model.device)
  
  # Perform text generation
  with torch.no_grad():
      generation_config = GenerationConfig(
          repetition_penalty=1.13,
          max_new_tokens=max_new_tokens,
          pad_token_id=tokenizer.pad_token_id,
          do_sample=False,
          use_cache=True,
      )
      generated = model.generate(
          inputs=input_ids,
          generation_config=generation_config,
      )
  
  # Get the generated tokens, starting from where the input ends
  gen_tokens = generated.cpu()[:, input_ids.shape[-1]:]  # Slice from the input length onward
  output = tokenizer.batch_decode(gen_tokens)[0]
  return output.replace("</s>", "").strip()

In [21]:
# --- RAG Pipeline ---
def rag_pipeline(question, topk=3):
  """
  End-to-end pipeline for retrieval-augmented generation (RAG).
  """
  # Retrieve relevant contexts
  top_passages = retrieve(question, topk=topk)
  # Generate a prompt
  prompt = get_prompt(question, top_passages)
  # Generate answer using the LLM
  generated_answer = generate(prompt)
  result = {
      "retrieved_context": top_passages,
      "generated_answer": generated_answer
  }
  return result

In [22]:
questions = [
    "Federated Learning là gì?",
    "Resnet18 là gì?",
    "Adversarial Training là gì?",
    " Tại sao lựa chọn Adversarial Training?"
]

# Step 4: Answer each question using RAG
for question in questions:
  print(f"\n\n{'='*50}")
  print(f"\nQuestion: {question}")
  result = rag_pipeline(question, topk=5)

  print("Generated Answer:")
  print(result["generated_answer"])




Question: Federated Learning là gì?
Generated Answer:
Federated Learning (FL) là một phương pháp học máy phân tán, nơi nhiều thiết bị hoặc nút tính toán cục bộ tham gia vào việc huấn luyện một mô hình chung mà không cần chia sẻ dữ liệu thô [1]. Nó giúp bảo vệ quyền riêng tư dữ liệu, giảm tải băng thông và cải thiện khả năng mở rộng hệ thống [1] [5]. Trong bài toán này, một biến thể của ResNet-18, một kiến trúc mạng nơ-ron tích chập (CNN) mạnh mẽ, được sử dụng làm trình trích xuất đặc trưng [1]. Mô hình gốc được tạo ra bằng cách huấn luyện mô hình cục bộ và gửi các tham số (weights) hoặc gradients đã học được về cho máy chủ trung tâm [1]. Sau đó, mô hình cập nhật lại được gửi lại cho các thiết bị để tiếp tục huấn luyện [1].



Question: Resnet18 là gì?
Generated Answer:
ResNet-18 là một kiến trúc mạng nơ-ron tích chập (CNN) mạnh mẽ, làm trình trích xuất đặc trưng và được sử dụng làm nền tảng cho mô hình LogisticRegressionModelV2 trong bài toán được đề cập. Nó được sử dụng để trích xu